In [1]:
import warnings
import sys
import os

warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
sys.path.append('/mnt/local/data/vtrifonov/prec-learning-Notay-loss/')

In [2]:
import jax.numpy as jnp
from jax import random, vmap, clear_caches, jit, device_put
import jax.experimental.sparse as jsparse

import ilupp
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt

import cloudpickle
from functools import partial
from time import perf_counter

from data.graph_utils import direc_graph_from_linear_system_sparse, bi_direc_indx
from linsolve.scipy_linsolve import solve_precChol, cg_scipy
from utils import jBCOO_to_scipyCSR, iter_per_residual

plt.rcParams['figure.figsize'] = (11, 7)

In [3]:
kaporin_ss = [
    'bodyy6','bcsstk18','bcsstk25','cvxbqp1','bcsstk17','gridgena',
    'apache1','Pres_Poisson','G2_circuit','olafu','gyro_k','msc23052',
    'bcsstk36','msc10848','raefsky4','cfd1','oilpan','vanbody','ct20stif',
    'nasasrb','cfd2','shipsec8','shipsec1','Dubcova3','parabolic_fem',
    's3dkt3m2','smt','ship_003','ship_001','cant','offshore','pdb1HYS',
    's3dkq4m2','thread','shipsec5','apache2','ecology2','tmt_sym','boneS01',
    'consph','bmw7st_1','G3_circuit','x104','thermal2','m_t1','hood','crankseg_1',
    'bmwcra_1','pwtk','crankseg_2','nd12k','af_shell7','msdoor','F1','nd24k','ldoor'
]
path = '/mnt/local/data/vtrifonov/susbet_SuiteSparse/scipy_sparse'

In [4]:
# Load model
save_path = '/mnt/local/data/vtrifonov/prec-learning-Notay-loss/trained_models/'
model_name = 'div_k_grad_128_0.7_ilu0_llt_loss_1000epoch'
with open(os.path.join(save_path, model_name + '.pkl'), 'rb') as f:
    model = cloudpickle.load(f)

In [5]:
def make_system(name, path):
    A = sparse.load_npz(os.path.join(path, name+'.npz')).tocsr()
    b = A @ np.ones(A.shape[0])
    return A, b

def make_ic0(A):
    s = perf_counter()
    L = ilupp.ichol0(A)
    t = perf_counter() - s 
    return sparse.linalg.LinearOperator(shape=L.shape, matvec=partial(solve_precChol, L=L)), t, L
    
def make_preccor_ic0(A, b, L_ic0, model):
    jA = device_put(jsparse.BCOO.from_scipy_sparse(A))
    jb = device_put(jnp.array(b, dtype=jnp.float32))
    jA_pad = device_put(jsparse.BCOO.from_scipy_sparse(L_ic0 + sparse.triu(L_ic0.T, k=1)).sort_indices())
    jit_model = jit(lambda t1, t2, t3, t4, t5: model((t1, t2, t3, t4), t5))
    
    
    nodes, edges, receivers, senders, n_node = direc_graph_from_linear_system_sparse(jA_pad[None, ...], jb[None, ...])
    bi_edges = bi_direc_indx(receivers[0, ...], senders[0, ...], n_node[1])
    L = jit_model(nodes[0, ...], edges[0, ...], receivers[0, ...], senders[0, ...], bi_edges)
    
    s = perf_counter()
    L = jit_model(nodes[0, ...], edges[0, ...], receivers[0, ...], senders[0, ...], bi_edges)
    t = perf_counter() - s 
    return sparse.linalg.LinearOperator(shape=L.shape, matvec=partial(solve_precChol, L=jBCOO_to_scipyCSR(L))), t

def jacobi_symm_scaling(A):
    D = sparse.diags(1 / np.sqrt(A.diagonal()))
    return D @ A @ D

def run_for_name(df, model, name, path, atol=1e-12, maxiter=2000, scaling=False, x0='random'):
    assert (x0 == 'random') | (x0 == None)
    A_i, b_i = make_system(name, path)
    x0 = np.array(random.normal(random.PRNGKey(42), b_i.shape)) if x0 == 'random' else None
    print(f'{name}, shape = {A_i.shape[0]}')
    
    if scaling:
        A_i = jacobi_symm_scaling(A_i)
    
    P_nla, t_nla, L_ic0_i = make_ic0(A_i)
    try: 
        P_dl, t_dl = make_preccor_ic0(A_i, b_i, L_ic0_i, model)
        _, res_dl, time_cg_dl = cg_scipy(A_i, b_i, P_dl, atol, maxiter)
    except Exception as e:
        t_dl, res_dl, time_cg_dl = -1, [np.nan], [np.nan]
        print(' Exception:', e)
        
    _, res_nla, time_cg_nla = cg_scipy(A_i, b_i, P_nla, atol, maxiter, x0)
    
    iters_nla = iter_per_residual(res_nla)
    iters_dl = iter_per_residual(res_dl)
    
    time_nla, time_dl = [], []
    for v in iters_nla.values():
        if np.isnan(v):
            time_nla.append(str(np.nan))
        else:
            time_nla.append(np.round(time_cg_nla[v], 5))
    
    for v in iters_dl.values():
        if np.isnan(v):
            time_dl.append(str(np.nan))
        else:
            time_dl.append(np.round(time_cg_dl[v], 5))
    
    cols = [
        'pre-time ic0', 'pre-time precor',
        'time ic0 1e-3', 'time precor 1e-3',
        'time ic0 1e-6', 'time precor 1e-6',
        'time ic0 1e-9', 'time precor 1e-9',
        'time ic0 1e-12', 'time precor 1e-12',
        'iter ic0 1e-3', 'iter precor 1e-3',
        'iter ic0 1e-6', 'iter precor 1e-6',
        'iter ic0 1e-9', 'iter precor 1e-9',
        'iter ic0 1e-12', 'iter precor 1e-12',
    ]
    vals = [
        np.round(t_nla, 6), np.round(t_dl, 6),
        time_nla[0], time_dl[0],
        time_nla[1], time_dl[1],
        time_nla[2], time_dl[2],
        time_nla[3], time_dl[3],
        iters_nla[1e-3], iters_dl[1e-3],
        iters_nla[1e-6], iters_dl[1e-6],
        iters_nla[1e-9], iters_dl[1e-9],
        iters_nla[1e-12], iters_dl[1e-12]
    ]
    for c, v in zip(cols, vals):
        df.loc[name, c] = str(v)

    return df

# With scaling and zero initial guess

In [6]:
len(kaporin_ss)

56

In [7]:
# Be careful!
df = pd.DataFrame({})

In [8]:
atol = 1e-9
maxiter = 5000
scaling = True
x0 = None

In [9]:
for name in kaporin_ss[:10]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

bodyy6, shape = 19366


2024-09-28 16:44:56.285158: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


bcsstk18, shape = 11948
bcsstk25, shape = 15439
cvxbqp1, shape = 50000
bcsstk17, shape = 10974
gridgena, shape = 48962
apache1, shape = 80800
Pres_Poisson, shape = 14822
G2_circuit, shape = 150102
olafu, shape = 16146


,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [10]:
for name in kaporin_ss[10:20]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

gyro_k, shape = 17361
msc23052, shape = 23052
bcsstk36, shape = 23052
msc10848, shape = 10848
raefsky4, shape = 19779
cfd1, shape = 70656
oilpan, shape = 73752
vanbody, shape = 47072
ct20stif, shape = 52329
nasasrb, shape = 54870


,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [11]:
for name in kaporin_ss[20:30]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

cfd2, shape = 123440
shipsec8, shape = 114919
shipsec1, shape = 140874
Dubcova3, shape = 146689
parabolic_fem, shape = 525825
s3dkt3m2, shape = 90449
smt, shape = 25710
ship_003, shape = 121728
ship_001, shape = 34920
cant, shape = 62451


,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [12]:
for name in kaporin_ss[30:40]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

offshore, shape = 259789
pdb1HYS, shape = 36417
s3dkq4m2, shape = 90449
thread, shape = 29736
shipsec5, shape = 179860
apache2, shape = 715176
ecology2, shape = 999999
tmt_sym, shape = 726713
boneS01, shape = 127224
consph, shape = 83334


,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [13]:
for name in kaporin_ss[40:50]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

bmw7st_1, shape = 141347
G3_circuit, shape = 1585478
x104, shape = 108384
thermal2, shape = 1228045
m_t1, shape = 97578
hood, shape = 220542
crankseg_1, shape = 52804
bmwcra_1, shape = 148770
pwtk, shape = 217918
crankseg_2, shape = 63838


,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
for name in kaporin_ss[50:]:
    df = run_for_name(df, model, name, path, atol=atol, maxiter=maxiter, scaling=scaling, x0=x0)    # x0 == None => x0 := zeros
df

nd12k, shape = 36000
af_shell7, shape = 504855
 Exception: A is singular: zero entry on diagonal.
msdoor, shape = 415863
 Exception: A is singular: zero entry on diagonal.
F1, shape = 343791
 Exception: A is singular: zero entry on diagonal.
nd24k, shape = 72000
 Exception: A is singular: zero entry on diagonal.


In [15]:
df

,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001249,0.002279,0.53394,nan,0.74215,nan,0.90049,nan,nan,nan,63,nan,88,nan,107,nan,nan,nan
bcsstk18,0.003413,0.002246,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.004906,0.002141,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.003829,0.002235,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.009051,0.002375,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.006883,0.002429,14.73118,nan,23.56734,nan,33.00739,nan,nan,nan,614,nan,976,nan,1356,nan,nan,nan
apache1,0.003455,0.002577,3.75247,nan,6.53471,nan,9.13869,nan,nan,nan,124,nan,216,nan,303,nan,nan,nan
Pres_Poisson,0.022272,0.002819,3.43212,nan,4.52527,nan,5.02581,nan,nan,nan,149,nan,197,nan,219,nan,nan,nan
G2_circuit,0.004799,0.002469,25.87714,nan,36.17725,nan,41.42856,nan,nan,nan,404,nan,565,nan,647,nan,nan,nan
olafu,0.028208,0.002898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [19]:
df.to_csv(os.path.join('/mnt/local/data/vtrifonov/susbet_SuiteSparse/results', '5000iter.csv'), index=True)

In [18]:
path

'/mnt/local/data/vtrifonov/susbet_SuiteSparse/scipy_sparse'

# First run with 500 iter max

In [6]:
# df = pd.DataFrame({})
for name in kaporin_ss:
    df = run_for_name(df, model, name, path, atol=1e-12, maxiter=500)

bodyy6, shape = 19366


2024-09-28 11:21:37.707997: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


bcsstk18, shape = 11948
bcsstk25, shape = 15439
cvxbqp1, shape = 50000
bcsstk17, shape = 10974
gridgena, shape = 48962
apache1, shape = 80800
Pres_Poisson, shape = 14822
G2_circuit, shape = 150102
olafu, shape = 16146
gyro_k, shape = 17361
msc23052, shape = 23052
bcsstk36, shape = 23052
msc10848, shape = 10848
raefsky4, shape = 19779
cfd1, shape = 70656
oilpan, shape = 73752
vanbody, shape = 47072
ct20stif, shape = 52329
nasasrb, shape = 54870
cfd2, shape = 123440
shipsec8, shape = 114919
shipsec1, shape = 140874
Dubcova3, shape = 146689
parabolic_fem, shape = 525825
s3dkt3m2, shape = 90449
smt, shape = 25710
ship_003, shape = 121728
ship_001, shape = 34920
cant, shape = 62451
offshore, shape = 259789
pdb1HYS, shape = 36417
s3dkq4m2, shape = 90449
thread, shape = 29736
shipsec5, shape = 179860
apache2, shape = 715176
ecology2, shape = 999999
tmt_sym, shape = 726713
boneS01, shape = 127224
consph, shape = 83334
bmw7st_1, shape = 141347
G3_circuit, shape = 1585478
x104, shape = 108384
th

LinAlgError: A is singular: zero entry on diagonal.

In [7]:
df

,pre-time ic0,pre-time precor,time ic0 1e-3,time precor 1e-3,time ic0 1e-6,time precor 1e-6,time ic0 1e-9,time precor 1e-9,time ic0 1e-12,time precor 1e-12,iter ic0 1e-3,iter precor 1e-3,iter ic0 1e-6,iter precor 1e-6,iter ic0 1e-9,iter precor 1e-9,iter ic0 1e-12,iter precor 1e-12
bodyy6,0.001041,0.003039,1.08859,nan,1.36662,nan,1.55887,nan,1.82216,nan,56,nan,82,nan,100,nan,124,nan
bcsstk18,0.004214,0.003459,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk25,0.00601,0.003531,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cvxbqp1,0.002055,0.003808,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
bcsstk17,0.008825,0.003989,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
gridgena,0.003844,0.003684,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
apache1,0.005531,0.003647,11.39554,nan,14.24843,nan,16.47388,nan,19.56842,nan,279,nan,352,nan,409,nan,488,nan
Pres_Poisson,0.021588,0.003781,4.99755,nan,6.4132,nan,7.06301,nan,7.623,nan,154,nan,202,nan,224,nan,243,nan
G2_circuit,0.004123,0.003756,24.52278,nan,nan,nan,nan,nan,nan,nan,391,nan,nan,nan,nan,nan,nan,nan
olafu,0.028611,0.004092,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [15]:
for name in ['af_shell7','msdoor','F1','nd24k','ldoor']:
    df = run_for_name(df, model, name, path, atol=1e-12, maxiter=500)

af_shell7, shape = 504855
 Exception: A is singular: zero entry on diagonal.
msdoor, shape = 415863
 Exception: A is singular: zero entry on diagonal.
F1, shape = 343791
 Exception: A is singular: zero entry on diagonal.
nd24k, shape = 72000
 Exception: A is singular: zero entry on diagonal.
ldoor, shape = 952203
 Exception: A is singular: zero entry on diagonal.


In [16]:
df.loc[:, ['iter ic0 1e-3', 'iter precor 1e-3']]

,iter ic0 1e-3,iter precor 1e-3
bodyy6,56,nan
bcsstk18,nan,nan
bcsstk25,nan,nan
cvxbqp1,nan,nan
bcsstk17,nan,nan
gridgena,nan,nan
apache1,279,nan
Pres_Poisson,154,nan
G2_circuit,391,nan
olafu,nan,nan
